https://www.pluralsight.com/guides/implement-hyperparameter-tuning-for-tensorflow2.0

In [3]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
from tqdm import tqdm
import glob
import numpy as np
import os
import shutil
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle

np.random.seed(42)
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

from keras.applications import vgg16
from keras.models import Model
import keras
import pandas as pd


from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from tensorflow.keras import optimizers

import glob
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.models import load_model
import os

os.chdir('/data/kanferg/Images/Pex_project/Transfer_learning/code')
from utils import Taining_data_orgenizer as orgenizer 
from utils import model_builder as mb

In [5]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [33]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [6]:
path_input = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/input_sc_mix'
path_origen = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set'

train_dir = os.path.join(path_origen, 'training_data')
val_dir = os.path.join(path_origen, 'validation_data')
test_dir = os.path.join(path_origen, 'test_data')

'''
batch size, epochs, steps_per_epoch_sel, validation_steps 
'''
# batch  = 30
# epoch  = 100
# step_per_epoch = int((9930)/30)
# validation_steps = int((1242)/30)
# path_model = '/data/kanferg/Images/Pex_project/Transfer_learning/models'
# # extract_size_train = 1000
# # extract_size_val = 200
# IMG_DIM=(150,150,3)
# imbalance_train = 921
# imbalance_val = 115


batch  = 30
epoch  = 100
step_per_epoch = 32
validation_steps = 10
path_model = '/data/kanferg/Images/Pex_project/Transfer_learning/models'
extract_size_train = 1000
extract_size_val = 200
IMG_DIM=(150,150,3)
imbalance_train = 921
imbalance_val = 115



'''
load image files (1000 for training and 200  for validation)
'''

model_build = mb.model_builder(IMG_DIM=(150,150,3),path_training=train_dir,path_validation=val_dir,
                 batch=batch, epoch = epoch,input_shape = (150,150,3) ,steps_per_epoch_sel= step_per_epoch,
                 validation_steps=validation_steps,path_model = path_model,file_extention = 'png',
                 imbalance_train = imbalance_train, imbalance_val = imbalance_val,extract_size_train=extract_size_train,extract_size_val=extract_size_val)
train_imgs_scaled, validation_imgs_scaled,train_labels_enc,validation_labels_enc,train_imgs,validation_imgs,report = model_build.build_image__sets()

run training


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:30<00:00, 32.84it/s]


run valadtion


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:06<00:00, 32.01it/s]


In [7]:
x_train = train_imgs_scaled
y_train = train_labels_enc
x_test  = validation_imgs_scaled
y_test = validation_labels_enc
print('train length {}  val length {}'.format(len(y_train),len(y_test)))

train length 1000  val length 200


In [37]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([256, 512]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.5, 0.6))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam','sgd','rmsprop']))
HP_L2 = hp.HParam('l2 regularizer', hp.RealInterval(.001,.01))

METRIC_ACCURACY = ['accuracy']

with tf.summary.create_file_writer('logs2/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER,HP_L2],
    metrics=[
            hp.Metric('accuracy', display_name='Accuracy'),
            hp.Metric('recall', display_name='Recall'),
            hp.Metric('specificity', display_name='Specificity'),
            hp.Metric('precision', display_name='Precision'),
            hp.Metric('f1score', display_name='F1Score'),
    ]
  
  )

In [38]:
#VGG16
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(input_shape=(150,150,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"),
        tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),
        tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),
        tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),
        tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], kernel_regularizer=tf.keras.regularizers.l2(0.001), activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(2, activation=tf.nn.softmax),
    ])
    model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=METRIC_ACCURACY,
    )

    model.fit(x_train, y_train, epochs=2) 
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

In [39]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY[0], accuracy, step=2)

In [40]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for l2 in (HP_L2.domain.min_value, HP_L2.domain.max_value):
      for optimizer in HP_OPTIMIZER.domain.values:
        hparams = {
            HP_NUM_UNITS: num_units,
            HP_DROPOUT: dropout_rate,
            HP_L2: l2,
            HP_OPTIMIZER: optimizer,
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs2/hparam_tuning/' + run_name, hparams)
        session_num += 1

--- Starting trial: run-0
{'num_units': 256, 'dropout': 0.5, 'l2 regularizer': 0.001, 'optimizer': 'adam'}
Epoch 1/2
32/32 [==============================] - 11s 328ms/step - loss: 0.7134 - accuracy: 0.7200
Epoch 2/2
7/7 [==============================] - 1s 93ms/step - loss: 0.1371 - accuracy: 0.9750
--- Starting trial: run-1
{'num_units': 256, 'dropout': 0.5, 'l2 regularizer': 0.001, 'optimizer': 'rmsprop'}
Epoch 1/2
32/32 [==============================] - 12s 337ms/step - loss: 3.6001 - accuracy: 0.5260
Epoch 2/2
7/7 [==============================] - 1s 94ms/step - loss: 0.2392 - accuracy: 0.9800
--- Starting trial: run-2
{'num_units': 256, 'dropout': 0.5, 'l2 regularizer': 0.001, 'optimizer': 'sgd'}
Epoch 1/2
32/32 [==============================] - 12s 337ms/step - loss: 1.2000 - accuracy: 0.5350
Epoch 2/2
7/7 [==============================] - 1s 98ms/step - loss: 1.1885 - accuracy: 0.9850
--- Starting trial: run-3
{'num_units': 256, 'dropout': 0.5, 'l2 regularizer': 0.01, 'opt

In [24]:
os.getcwd()

'/gpfs/gsfs10/users/kanferg/Images/Pex_project/Transfer_learning/code'

In [26]:
"run-%d" % session_num
session_num += 1

In [27]:
"run-%d" % session_num

'run-1'

In [28]:
hparams

{HParam(name='num_units', domain=Discrete([256, 512]), display_name=None, description=None): 256,
 HParam(name='dropout', domain=RealInterval(0.5, 0.6), display_name=None, description=None): 0.5,
 HParam(name='l2 regularizer', domain=RealInterval(0.001, 0.01), display_name=None, description=None): 0.001,
 HParam(name='optimizer', domain=Discrete(['adam', 'rmsprop', 'sgd']), display_name=None, description=None): 'adam'}

In [29]:
METRIC_ACCURACY

['accuracy']

In [30]:
hp.hparams(hparams)  # record the values used in this trial


<tf.Tensor: shape=(), dtype=bool, numpy=False>

In [31]:
accuracy = train_test_model(hparams)


Epoch 1/2
32/32 [==============================] - 12s 327ms/step - loss: 0.7986 - accuracy: 0.7060
Epoch 2/2
7/7 [==============================] - 1s 93ms/step - loss: 0.1782 - accuracy: 0.9800


In [32]:
tf.summary.scalar(METRIC_ACCURACY, accuracy, step=2)

TypeError: expected string or bytes-like object

In [33]:
METRIC_ACCURACY

['accuracy']

In [ ]:
os.getcwd